## The purpose of this notebook is to evaluate the discriminative capabilities of the representations learned by a post-hoc encoder trained with a fixed generator by performing keyword detection on the SC09 dataset

* We train a Logistic Regression as well as a Random Forest classifier on top of those features and report our performance on the SC09 test set.

### Import packages

In [1]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score

### Define paths

In [4]:
data_dir = os.path.join("/media", "datastore", "c-matsty-data", "datasets", "SpeechCommands")

#### Discriminator features path

In [6]:
encoder_features_dirname = "Posthoc_encoder_features"
training_input_path = os.path.join(data_dir, encoder_features_dirname, "training.npy")
test_input_path = os.path.join(data_dir, encoder_features_dirname, "test.npy")

#### Label paths

In [7]:
training_label_dir = os.path.join(data_dir, "SpeechCommands_Preproc_2_training", "labels")
test_label_dir = os.path.join(data_dir, "SpeechCommands_Preproc_2_test", "labels")

In [8]:
def load_labels(labels_path):
    Y = []
    for input_file_name in os.listdir(labels_path):
        y = np.load(os.path.join(labels_path, input_file_name))[..., np.newaxis]
        Y.append(y)
    Y = np.vstack(Y)
    return Y

### Load data

In [27]:
X_tr = np.load(training_input_path)

In [28]:
X_ts = np.load(test_input_path)

In [29]:
y_tr = load_labels(training_label_dir)
y_ts = load_labels(test_label_dir)

In [30]:
label_dict = {value: index  for index, value in enumerate(np.unique(y_tr))}

### Prepare data for training

#### Turn labels from strings to integer identifiers

In [31]:
y_tr = np.vectorize(label_dict.get)(y_tr)
y_ts = np.vectorize(label_dict.get)(y_ts)

#### Reshape input arrays and labels

In [32]:
y_tr = y_tr.flatten()
y_ts = y_ts.flatten()

### Normalize data

In [16]:
mean = X_tr.mean()
std = X_tr.std()
X_tr = (X_tr - mean) / std 
X_ts = (X_ts - mean) / std

### Define sample weights

In [33]:
class_counts = [len(y_tr[y_tr == i]) for i in label_dict.values()]
class_weights = [max(class_counts)/class_count for class_count in class_counts]
class_weight_dict = {class_idx: class_weight for class_idx, class_weight in zip(label_dict.values(), class_weights)}

In [34]:
label_dict

{'eight': 0,
 'five': 1,
 'four': 2,
 'nine': 3,
 'one': 4,
 'seven': 5,
 'six': 6,
 'three': 7,
 'two': 8,
 'zero': 9}

### Train and test logistic regression classifier

In [42]:
multi_class = 'multinomial'
model = LogisticRegression(multi_class=multi_class, max_iter=15000, random_state=0, penalty='elasticnet', solver='saga', l1_ratio=0.5)

In [43]:
sample_weight = [class_weight_dict[label] for label in y_tr]

In [44]:
model = model.fit(X_tr, y_tr, sample_weight=sample_weight)

In [45]:
y_preds = model.predict(X_ts)
print(classification_report(y_ts, y_preds))
print(balanced_accuracy_score(y_ts, y_preds))

              precision    recall  f1-score   support

           0       0.04      0.04      0.04       257
           1       0.09      0.11      0.10       270
           2       0.08      0.10      0.09       253
           3       0.12      0.11      0.11       259
           4       0.06      0.07      0.06       248
           5       0.09      0.08      0.08       239
           6       0.11      0.09      0.10       244
           7       0.15      0.12      0.13       267
           8       0.08      0.08      0.08       264
           9       0.05      0.05      0.05       250

    accuracy                           0.09      2551
   macro avg       0.09      0.09      0.09      2551
weighted avg       0.09      0.09      0.09      2551

0.08505391982672493


### Train and test RandomForest classifier

In [46]:
model = RandomForestClassifier(n_estimators=100, random_state=0)

In [47]:
model = model.fit(X_tr, y_tr, sample_weight=sample_weight)

In [48]:
y_preds = model.predict(X_ts)
print(classification_report(y_ts, y_preds))
print(balanced_accuracy_score(y_ts, y_preds))

              precision    recall  f1-score   support

           0       0.09      0.09      0.09       257
           1       0.10      0.11      0.11       270
           2       0.09      0.09      0.09       253
           3       0.07      0.07      0.07       259
           4       0.06      0.08      0.07       248
           5       0.08      0.08      0.08       239
           6       0.08      0.07      0.07       244
           7       0.06      0.04      0.05       267
           8       0.10      0.08      0.09       264
           9       0.06      0.06      0.06       250

    accuracy                           0.08      2551
   macro avg       0.08      0.08      0.08      2551
weighted avg       0.08      0.08      0.08      2551

0.07831317614203837
